# Homework II

### Due Date: 11:55 PM Thursday, March 1, 2018

You should turn in the notebook at Columbia CourseWorks website.

Before you turn in the notebook, press the "Run all cells" button in the toolbar, and make sure all the calculation results and graphs are produced correctly, and then save the notebook.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

# Dual Problem and Upper Bounds for American Options

<strong>Primal Problem</strong> (maximizing over stopping times)

\begin{equation*}
V_t=\sup_{\tau\in\mathcal{T}_{t, T}}E^Q\left[D_{t, \tau}F_{\tau}\big|\mathcal{F}_t\right]
\end{equation*}

<strong>Dual Problem</strong> (minimizing over martingales) [Rogers 2002; Haugh and Kogan 2004]

\begin{equation*}
V_t=\inf_{M\in\mathcal{M}_{t, 0}}E^Q\left[\sup_{t\leq s\leq T}\left(D_{t, s}F_s-M_s\right)\bigg|\mathcal{F}_t\right].
\end{equation*}

where $\mathcal{M}_{t, 0}$ denotes the set of all right-continuous martingales $(M_s, s\in[t, T])$ with $M_t=0$.
The optimal martingale $M^{\ast}$ is the martingale part of the Doob-Meyer decomposition of $(S_s-S_t)/D_{0, t}$, $t\leq s\leq T$.

In particular, any martingale $M_s$ with $M_0=0$ gives an upper bound for the price at time 0, $V_0$:

$$V_0 \leq\mathbb{E}^{\mathbb{Q}}\left[\sup_{0\leq s\leq T}\left(D_sF_s-M_s\right)\right].$$

<b style="color:darkorange">Question 1.</b> Consider pricing a one-year Bermudan put option with monthly exercise, where the asset price process is assumed to follow geometric Brownian motion with $S_0=100$, $\sigma=0.2$, $r=0.1$, $q=0.02$, $K=100$ and exercise dates $t_1=\frac{1}{12}$, $t_2=\frac{2}{12}$, $\cdots$, $t_{12}=1$.

Simulate 100,000 paths and use the following martingale to find a upper bound.

(a). $M_t\equiv0$.

(b). $M_t$ is the discounted European put price with the same final maturity less the initial price.

For your reference, the price of the option is $5.155$.

In [2]:
def blackscholes_mc(S=100, vol=0.2, r=0, q=0, ts=np.linspace(0, 1, 13), npaths=10):
    """Generate Monte-Carlo paths in Black-Scholes model.

    Parameters
    ----------
    S: scalar
        The spot price of the underlying security.
    vol: scalar
        The implied Black-Scholes volatility.
    r: scalar
        The annualized risk-free interest rate, continuously compounded.
    q: scalar
        The annualized continuous dividend yield.
    ts: array_like
        The time steps of the simualtion
    npaths: int
        the number of paths to simulate

    Returns
    -------
    paths: ndarray
        The Monte-Carlo paths.
    """
    nsteps = len(ts) - 1
    ts = np.asfarray(ts)[:, np.newaxis]
    W = np.cumsum(np.vstack((np.zeros((1, npaths), dtype=np.float),
                             np.random.randn(nsteps, npaths) * np.sqrt(np.diff(ts, axis=0)))),
                  axis=0)
    paths = np.exp(-0.5*vol**2*ts + vol*W)*S*np.exp((r-q)*ts)
    return paths


def blackscholes_price(K, T, S, vol, r=0, q=0, callput='call'):
    """Compute the call/put option price in the Black-Scholes model
    
    Parameters
    ----------
    K: scalar or array_like
        The strike of the option.
    T: scalar or array_like
        The maturity of the option, expressed in years (e.g. 0.25 for 3-month and 2 for 2 years)
    S: scalar or array_like
        The current price of the underlying asset.
    vol: scalar or array_like
        The implied Black-Scholes volatility.
    r: scalar or array_like
        The annualized risk-free interest rate, continuously compounded.
    q: scalar or array_like
        The annualized continuous dividend yield.
    callput: str
        Must be either 'call' or 'put'.

    Returns
    -------
    price: scalar or array_like
        The price of the option.

    Examples
    --------
    >>> blackscholes_price(95, 0.25, 100, 0.2, r=0.05, callput='put')
    1.5342604771222823
    """
    if T==0:
        return np.maximum(K-S, 0)
    else:
        F = S*np.exp((r-q)*T)
        v = np.sqrt(vol**2*T)
        d1 = np.log(F/K)/v + 0.5*v
        d2 = d1 - v
        try:
            opttype = {'call':1, 'put':-1}[callput.lower()]
        except:
            raise ValueError('The value of callput must be either "call" or "put".')
        price = opttype*(F*norm.cdf(opttype*d1)-K*norm.cdf(opttype*d2))*np.exp(-r*T)
        return price

<b>Question 1 Result</b>

In [3]:
S = 100
K = 100
vol = 0.2
r = 0.1
q = 0.02
ts = np.linspace(0, 1, 13)
npaths = 100000
paths = blackscholes_mc(S, vol, r, q, ts, npaths)
payoff1 = np.zeros((len(ts)-1,npaths), dtype=np.float)
payoff2 = np.zeros((len(ts)-1,npaths), dtype=np.float)
p0 = blackscholes_price(K, ts[-1], S, vol, r, q, callput='put')
for i in range(1, len(ts)):
    exval = np.maximum(K-paths[i], 0)
    bsprice = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, r, q, callput='put')
    payoff1[i-1] = exval*np.exp(-r*ts[i])
    payoff2[i-1] = (exval - bsprice)*np.exp(-r*ts[i]) + p0
    
V0 = {'No Hedge': np.mean(np.max(payoff1, 0)), 'BS Price Hedge': np.mean(np.max(payoff2, 0))}
V0

{'BS Price Hedge': 5.354062848847926, 'No Hedge': 8.6229022590723421}

<b>Question 1 Comment</b>
<ul>
<li>With hedging, the upper bound for the option price is effectively reduced.
</ul>

## Optimal martingale

The optimal martingale that gives zero <em>duality gap</em> is the martingale component of the discounted value process (known as Snell envelope) $\frac{V_t}{D_{0,t}}$, following the optimal exercise strategy.

To get a good upper bound, we first find a value process that is close to the true value process, and then extract its martingale component. This approximation value process can be obtained from a functional approximation such as regression, or from an exercise strategy (stopping time).

In general, for a stochastic process $\{U_n\}_{n\geq0}$, to extract its martingale component, we only need to let $M_0=0$, and then

$$M_{n+1}-M_n=U_{n+1}-E\left[U_{n+1}\big|\mathcal{F}_n\right],\quad n=0,1,\cdots.$$

As a demonstration, we show below the code that calculates the upper bound estimate of the price of Bermudan put with monthly exercises as in Question 1.1. The optimal exercise frontier is given for each exercise date (from a PDE solver), i.e. if the underlying price falls below Bs[i] at time ts[i], then we should exercise the option, otherwise we should continue.

In [4]:
Bs = np.array([87.527, 87.778, 88.056, 88.369, 88.724, 89.131, 89.605, 90.170, 90.862, 91.745, 92.952, 94.849, 100])

def exer_or_cont(i, S):
    return S <= Bs[i]

<b>Lower Bound.</b>
Simulate independent paths and exercise the option acoording to this strategy

In [5]:
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)

npaths = 1000000
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
alive = np.full(npaths, True, dtype=bool)
payoff = np.zeros(npaths, dtype=np.float)
for i in range(1, len(ts)):
    exerval = np.maximum(K-paths[i], 0)
    ind = exer_or_cont(i, paths[i]) & alive
    payoff[ind] = exerval[ind]*np.exp(-r*ts[i])
    alive[ind] = False
V0 = np.mean(payoff)
V0

5.1450781902635159

<b>Upper Bound.</b> We use nested simulation to estimate $V_n$ and $E\left[V_{n+1}\big|\mathcal{F}_n\right]$, where $V_n$ is the discounted value at time $t_n$.

In [6]:
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)

# nested simulation from time t_i when stock price is S
def nested_mc(i, S, nsims=1000):
    alive = np.arange(nsims)                                   # keep track of indices of paths still alive
    payoff = np.zeros(nsims, dtype=np.float)
    paths = np.full(nsims, S, dtype=np.float)
    for j in range(i+1, len(ts)):
        if len(alive) == 0:                                    # if exercised for all paths, stop
            break
        dt = ts[j]-ts[j-1]
        dW = np.random.randn(len(alive))*np.sqrt(dt)           # Brownian increment
        paths[alive] = paths[alive] * np.exp(-0.5*vol**2*dt+vol*dW)*np.exp((r-q)*dt)  # simulate S_{j+1} from S_j
        ind = exer_or_cont(j, paths[alive])                    # identify the paths that need exercise
        payoff[alive[ind]] = np.maximum(K-paths[alive[ind]], 0)*np.exp(-r*ts[j])      # update payoffs for exercised paths
        alive = alive[~ind]                                    # remove exercised paths 
    return np.mean(payoff)                                     # taking average of paths

# Upper bound
npaths = 1000                                                  # number of paths in the second independent run
nsims = 500                                                    # number of paths used in nested simulation
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
pathsV = np.empty(paths.shape, dtype=np.float)                 # discounted value process V_i (discounted to time zero)
pathsV[0] = V0                                                 # Initial value from lower bound simulation
pathsV[-1] = np.maximum(K-paths[-1], 0)*np.exp(-r*ts[-1])      # values at final maturity
pathsEV = np.empty(paths.shape, dtype=np.float)                # Conditional expectation E[V_{i+1}|F_i], or continuation value
pathsEV[0] = pathsV[0]                                         # at time 0, option value = continuation value
for i in range(1, len(ts)-1):
    for j in range(npaths):
        S = paths[i, j]                                        # stock price at time t_i and j-th path
        if exer_or_cont(i, S):
            pathsV[i, j] = np.maximum(K-S, 0)*np.exp(-r*ts[i]) # if exercised, V_i = exercise value
            pathsEV[i, j] = nested_mc(i, S, nsims=nsims)       # launch nested simulation to estimate E[V_{i+1}|F_i]
        else:
            pathsV[i, j] = nested_mc(i, S, nsims=nsims)        # if continue, use nested simulation to estimate V_i
            pathsEV[i, j] = pathsV[i, j]                       # E[V_{i+1}|F_i] = V_i
M = np.zeros_like(paths)
M[1:] = np.cumsum(pathsV[1:]-pathsEV[:-1], axis=0)             # martinglae increment V_{i+1}-E[V_{i+1}|F_i]
# taking average of max value of D_sF_s-M_s along each path
np.mean((np.maximum(K-paths, 0)*np.exp(-r*ts[:, np.newaxis])-M).max(axis=0))

5.1599555685621326

Since the strategy is optimal, we see that both lower bound and upper bound prices converge to the true value. 

However, in general, we may only get a sub-optimal strategy (e.g. from Longstaff-Schwartz algorithm). But with this sub-optimal strategy, we can build the (discounted) value process and then extract its martingale component to obtain an upper bound. Intuitively speaking, the closer this sub-optimal strategy is to the optimal one, the tighter the corresponding upper bound.

<b style="color:darkorange">Question 2.</b> Consider pricing the Bermudan put option as in Question 1.1 by using the constant $1.0$ and the Black-Scholes price of a European put option with volatility $0.2$ and maturity $T-t_n$ as the two basis functions at time $t_n$. Use the Longstaff-Schwartz algorithm to build an exercise strategy with these basis functions and then compute the corresponding upper bound.

For your convenience, an implementation of Longstaff-Schwartz algorithm is included in the cell below. It computes the regression coefficients at each exercise date.

In [7]:
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)

npaths = 10000
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
betas_LS = np.full((len(ts), 2), np.nan, dtype=np.float)
payoff = np.maximum(K-paths[-1], 0)
for i in range(len(ts)-2, 0, -1):
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    payoff = payoff*discount
    Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, r, q, callput='put')
    A = np.vstack((np.ones_like(Z), Z)).T
    betas_LS[i] = np.linalg.lstsq(A, payoff)[0]    # regression to estimate continuation values
    contval = betas_LS[i, 0]+betas_LS[i, 1]*Z
    exerval = np.maximum(K-paths[i], 0)
    # identify the paths where we should exercise
    ind = exerval > contval
    payoff[ind] = exerval[ind]                     # update payoff on exercised paths
betas_LS                                           # No regression needed at first and last time steps, return NaN

array([[        nan,         nan],
       [-0.44553251,  1.28378532],
       [-0.40848886,  1.27628617],
       [-0.31953967,  1.24413462],
       [-0.206712  ,  1.20454894],
       [-0.20523879,  1.1934065 ],
       [-0.14715557,  1.16515974],
       [-0.11897509,  1.13015644],
       [-0.03035003,  1.08788336],
       [-0.03837493,  1.07497193],
       [-0.03600466,  1.04018726],
       [-0.01492042,  1.0027343 ],
       [        nan,         nan]])

<b>Question 2 Result</b>

In [8]:
# Longstaff_Schwartz optimal exercise strategy
def new_exer_or_cont(i, S):
    if i >= len(ts)-1:
        return K>S  #at maturity, exercise only if strike is larger than stock price
    else:
        Z = blackscholes_price(K, ts[-1]-ts[i], S, vol, r, q, callput='put')
        contval = betas_LS[i, 0]+betas_LS[i, 1]*Z
        exerval = np.maximum(K-S, 0)
        return exerval>contval  # exercise only if exercise value is larger than continuation value
    
# nested simulation from time t_i when stock price is S
def new_nested_mc(i, S, nsims=1000):
    alive = np.arange(nsims)                                   # keep track of indices of paths still alive
    payoff = np.zeros(nsims, dtype=np.float)
    paths = np.full(nsims, S, dtype=np.float)
    for j in range(i+1, len(ts)):
        if len(alive) == 0:                                    # if exercised for all paths, stop
            break
        dt = ts[j]-ts[j-1]
        dW = np.random.randn(len(alive))*np.sqrt(dt)           # Brownian increment
        paths[alive] = paths[alive] * np.exp(-0.5*vol**2*dt+vol*dW)*np.exp((r-q)*dt)  # simulate S_{j+1} from S_j
        ind = new_exer_or_cont(j, paths[alive])                    # identify the paths that need exercise
        payoff[alive[ind]] = np.maximum(K-paths[alive[ind]], 0)*np.exp(-r*ts[j])      # update payoffs for exercised paths
        alive = alive[~ind]                                    # remove exercised paths 
    return np.mean(payoff)                                     # taking average of paths



# Upper bound
npaths = 1000                                                  # number of paths in the second independent run
nsims = 500                                                    # number of paths used in nested simulation
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
pathsV = np.empty(paths.shape, dtype=np.float)                 # discounted value process V_i (discounted to time zero)
pathsV[0] = V0                                                 # Initial value from lower bound simulation
pathsV[-1] = np.maximum(K-paths[-1], 0)*np.exp(-r*ts[-1])      # values at final maturity
pathsEV = np.empty(paths.shape, dtype=np.float)                # Conditional expectation E[V_{i+1}|F_i], or continuation value
pathsEV[0] = pathsV[0]                                         # at time 0, option value = continuation value
for i in range(1, len(ts)-1):
    for j in range(npaths):
        S = paths[i, j]                                        # stock price at time t_i and j-th path
        if new_exer_or_cont(i, S):
            pathsV[i, j] = np.maximum(K-S, 0)*np.exp(-r*ts[i]) # if exercised, V_i = exercise value
            pathsEV[i, j] = new_nested_mc(i, S, nsims=nsims)       # launch nested simulation to estimate E[V_{i+1}|F_i]
        else:
            pathsV[i, j] = new_nested_mc(i, S, nsims=nsims)        # if continue, use nested simulation to estimate V_i
            pathsEV[i, j] = pathsV[i, j]                       # E[V_{i+1}|F_i] = V_i
M = np.zeros_like(paths)
M[1:] = np.cumsum(pathsV[1:]-pathsEV[:-1], axis=0)             # martinglae increment V_{i+1}-E[V_{i+1}|F_i]
# taking average of max value of D_sF_s-M_s along each path
np.mean((np.maximum(K-paths, 0)*np.exp(-r*ts[:, np.newaxis])-M).max(axis=0))

5.1760693355556411

<b>Question 2 Comment</b>
<ul>
<li>Using Longstaff-Schwartz optimal exercise strategy, the upper bound is very close to the upper bound using optimal exercise frontier. 
</ul>

<b style="color:darkorange">Question 3.</b> <b>(An alternative algorithm to estimate the upper bound)</b> The primal algorithm (Longstaff-Schwartz or Tsitsiklis-Van Roy) provides a function approximation (via regression) of the continuation value $C_n$ at $t_n$, hence of the value function $V_n = \max(C_n,F_n)$ where $F_n$ is the exercise value. We can use this functional approximation of the value function to write an alternative algorithm in order to estimate the upper bound: at each time $t_n$, use the functional approximation of $V_n$, and run the nested simulation to estimate the conditional expectation $E\left[V_{n+1}\big|\mathcal{F}_n\right]$ (using the functional approximation of $V_{n+1}$). Note that this nested simulation only needs to be run for one time step.

For the same Bermudan put option and same basis functions, use TVR algorithm to build the value process and then extract its martingale component using the alternative algorithm explained above to estimate the upper bound price.

For your convenience, an implementation of TVR algorithm is included in the cell below. It computes the regression coefficients at each exercise date. 

In [9]:
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)

npaths = 100000
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
betas_TVR = np.full((len(ts), 2), np.nan, dtype=np.float)
V = np.maximum(K-paths[-1], 0)
for i in range(len(ts)-2, 0, -1):
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, r, q, callput='put')
    A = np.vstack((np.ones_like(Z), Z)).T
    betas_TVR[i] = np.linalg.lstsq(A, V*discount)[0]      # regression to estimate continuation values
    contval = betas_TVR[i, 0]+betas_TVR[i, 1]*Z
    exerval = np.maximum(K-paths[i], 0)
    V = np.maximum(exerval, contval)                      # compute values
betas_TVR                                                 # No regression needed at first and last time steps, return NaN

array([[        nan,         nan],
       [-0.43111833,  1.30841043],
       [-0.38471921,  1.29163439],
       [-0.28696651,  1.2624234 ],
       [-0.19511503,  1.22860281],
       [-0.1176159 ,  1.1959613 ],
       [-0.06147771,  1.16268112],
       [-0.02976586,  1.13067906],
       [-0.00260358,  1.09804626],
       [ 0.01512692,  1.06593805],
       [ 0.008383  ,  1.03054199],
       [-0.00613147,  1.00059086],
       [        nan,         nan]])

<b>Question 3 Result</b>

In [10]:
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)

# Functional approximation via TVR algorithm
def func_approximation_TVR(i, S):
    if i >= len(ts)-1:
        return np.maximum(K-S,0)*np.exp(-r*ts[i])
    else:
        Z = blackscholes_price(K, ts[-1]-ts[i], S, vol, r, q, callput='put')
        contval = betas_TVR[i, 0]+betas_TVR[i, 1]*Z
        exerval = np.maximum(K-S, 0)
        V = np.maximum(contval, exerval)
        return V*np.exp(-r*ts[i])  # functional approximation of V_i
    
# nested simulation for one time step from time t_i when stock price is S
def nested_mc_TVR(i, S, nsims):
    payoff = np.zeros(nsims, dtype=np.float)
    dt = ts[i+1]-ts[i]
    dW = np.random.randn(nsims)*np.sqrt(dt)           # Brownian increment
    paths = S * np.exp(-0.5*vol**2*dt+vol*dW)*np.exp((r-q)*dt)  # simulate S_{i+1} from S_i
    payoff = func_approximation_TVR(i+1, paths)                                     
    return np.mean(payoff)                                     # taking average of paths


# Upper bound
npaths = 1000                                                  # number of paths in the second independent run
nsims = 500                                                    # number of paths used in nested simulation
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
pathsV = np.empty(paths.shape, dtype=np.float)                 # discounted value process V_i (discounted to time zero)
pathsV[0] = V0                                                 # Initial value from lower bound simulation
pathsV[-1] = np.maximum(K-paths[-1], 0)*np.exp(-r*ts[-1])      # values at final maturity
pathsEV = np.empty(paths.shape, dtype=np.float)                # Conditional expectation E[V_{i+1}|F_i], or continuation value
pathsEV[0] = pathsV[0]                                         # at time 0, option value = continuation value
for i in range(1, len(ts)-1):
    for j in range(npaths):
        S = paths[i, j]                                        # stock price at time t_i and j-th path
        pathsV[i, j] = func_approximation_TVR(i, S)            # use functional approximation to estimate V_i
        pathsEV[i, j] = nested_mc_TVR(i, S, nsims)             # use nested simulation to estimate E[V_{i+1}|F_i]
M = np.zeros_like(paths)
M[1:] = np.cumsum(pathsV[1:]-pathsEV[:-1], axis=0)             # martinglae increment V_{i+1}-E[V_{i+1}|F_i]
# taking average of max value of D_sF_s-M_s along each path
np.mean((np.maximum(K-paths, 0)*np.exp(-r*ts[:, np.newaxis])-M).max(axis=0))

5.0576091448150873

In [11]:
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)

# Functional approximation via LS algorithm
def func_approximation_LS(i, S):
    if i >= len(ts)-1:
        return np.maximum(K-S,0)*np.exp(-r*ts[i])
    else:
        Z = blackscholes_price(K, ts[-1]-ts[i], S, vol, r, q, callput='put')
        contval = betas_LS[i, 0]+betas_LS[i, 1]*Z
        exerval = np.maximum(K-S, 0)
        V = np.maximum(contval, exerval)
        return V*np.exp(-r*ts[i])  # functional approximation of V_i

        
# nested simulation for one time step from time t_i when stock price is S
def nested_mc_LS(i, S, nsims):
    payoff = np.zeros(nsims, dtype=np.float)
    dt = ts[i+1]-ts[i]
    dW = np.random.randn(nsims)*np.sqrt(dt)           # Brownian increment
    paths = S * np.exp(-0.5*vol**2*dt+vol*dW)*np.exp((r-q)*dt)  # simulate S_{i+1} from S_i
    payoff = func_approximation_LS(i+1, paths)                                     
    return np.mean(payoff)                                     # taking average of paths


# Upper bound
npaths = 1000                                                  # number of paths in the second independent run
nsims = 500                                                    # number of paths used in nested simulation
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
pathsV = np.empty(paths.shape, dtype=np.float)                 # discounted value process V_i (discounted to time zero)
pathsV[0] = V0                                                 # Initial value from lower bound simulation
pathsV[-1] = np.maximum(K-paths[-1], 0)*np.exp(-r*ts[-1])      # values at final maturity
pathsEV = np.empty(paths.shape, dtype=np.float)                # Conditional expectation E[V_{i+1}|F_i], or continuation value
pathsEV[0] = pathsV[0]                                         # at time 0, option value = continuation value
for i in range(1, len(ts)-1):
    for j in range(npaths):
        S = paths[i, j]                                        # stock price at time t_i and j-th path
        pathsV[i, j] = func_approximation_LS(i, S)            # use functional approximation to estimate V_i
        pathsEV[i, j] = nested_mc_LS(i, S, nsims)             # use nested simulation to estimate E[V_{i+1}|F_i]
M = np.zeros_like(paths)
M[1:] = np.cumsum(pathsV[1:]-pathsEV[:-1], axis=0)             # martinglae increment V_{i+1}-E[V_{i+1}|F_i]
# taking average of max value of D_sF_s-M_s along each path
np.mean((np.maximum(K-paths, 0)*np.exp(-r*ts[:, np.newaxis])-M).max(axis=0))

5.1989590787059932

<b>Question 3 Comment</b>
<ul> 
<li> I obtain the optimal martingale from functional approximation through TVR algorithm and LS algorithm respectively. The functional approximation strategy via LS algorithm obtains an actual upper bound, though it doesn't do a better job than LS optimal exercise strategy. However, the functional approximation strategy via TVR algorithm gets a upper bound even smaller than the lower bound, partly because of the bias propagated through TVR algorithm 
</ul>

The upper bound estimator can be used to check if the basis functions used in regression are adequate. A tight duality gap between the lower bound and upper bound is a confirmation that the linear span of the selected basis functions indeed gives a good approximation to the continuation value.

The upper bound estimator can be used to check if the basis functions used in regression are adequate. A tight duality gap between the lower bound and upper bound is a confirmation that the linear span of the selected basis functions indeed gives a good approximation to the continuation value. We now illustrate this with the pricing of Bermudan-Asian call options.

<b style="color:darkorange">Question 4</b> (<b>Bermudan-Asian Call Option</b>). In Question 2.5 of Homework I, we priced a Bermudan-Asian call option using a particular set of basis functions. Assume that all the parameters stay the same.

<b>(a).</b> Reuse the exercise strategy derived from the basis functions used in Question 2.5 of Homework I by Longstaff-Schwartz with $\bar{\sigma}=0.1$ to estimate the lower and upper bound. For your convenience, an implementation of Longstaff-Schwartz is included in the cell below. Note that the lower bound should be obtained by running new independent paths stopped according to the strategy from the Longstaff-Schwartz algorithm.

<b>(b).</b> As an alternative set of basis functions at time $t_n$, use 
$$1, A_{t_n}, S_{t_n}, A_{t_n}^2, S_{t_n}^2, A_{t_n}S_{t_n}.$$
in the Longstaff-Schwartz algorithm and then give the corresponding lower bound and upper bound.

<b>(c).</b> Compare the duality gaps obtained from (a) and (b). Which set of basis functions does a better job at estimating the optimal exercise strategy?

In [12]:
# Longstaff-Schwartz
S0, K, vol, r, q = 100, 100, 0.2, 0, 0
ts = np.linspace(0, 1, 13)
npaths = 100000
pathsS = blackscholes_mc(S0, vol, r, q, ts=ts, npaths=npaths)                                  # stock prices
pathsA = np.vstack((pathsS[0], np.cumsum(pathsS[1:], axis=0)/np.arange(1, 13)[:, np.newaxis])) # running averages
vol_ = 0.1
payoff = np.maximum(pathsA[-1]-K, 0)
betas_AC = np.full((len(ts), 2), np.nan, dtype=np.float)
for i in range(len(ts)-2, 0, -1):
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    payoff = payoff*discount
    Z = (i*pathsA[i]+(12-i)*pathsS[i])/12
    callZ = blackscholes_price(K, ts[-1]-ts[i], Z, vol_, callput='call')
    A = np.vstack((np.ones_like(Z), callZ)).T
    betas_AC[i] = np.linalg.lstsq(A, payoff)[0]           # regression to estimate continuation values
    contval = betas_AC[i, 0]+betas_AC[i, 1]*callZ
    exerval = np.maximum(pathsA[i]-K, 0)
    ind = exerval > contval                               # identify the paths where we should exercise
    payoff[ind] = exerval[ind]
betas_AC

array([[        nan,         nan],
       [ 0.27750785,  1.13478754],
       [-0.02082635,  1.11508281],
       [-0.19544691,  1.09281987],
       [-0.30036117,  1.07561721],
       [-0.34313518,  1.05727099],
       [-0.38581098,  1.04969409],
       [-0.38836473,  1.0403536 ],
       [-0.36448903,  1.03234875],
       [-0.31503647,  1.02436912],
       [-0.25806363,  1.01769945],
       [-0.17153641,  1.00835075],
       [        nan,         nan]])

<b>Question 4(a) Lower Bound</b>

In [13]:
S0, K, vol, r, q, vol_ = 100, 100, 0.2, 0, 0, 0.1
ts = np.linspace(0, 1, 13)

# Optimal stopping strategy
def ac_exer_or_cont(i,cumS, S):
    if i>=len(ts)-1:
        return cumS>K
    else:
        Z = (i*cumS+(12-i)*S)/12
        callZ = blackscholes_price(K, ts[-1]-ts[i], Z, vol_, callput='call')
        contval = betas_AC[i, 0]+betas_AC[i, 1]*callZ
        exerval = np.maximum(cumS-K, 0)
        return exerval>contval  # exercise only if exercise value is larger than continuation value

# Lower Bound
npaths = 1000000
pathsS1 = blackscholes_mc(S0, vol, r, q, ts, npaths=npaths)                                  # stock prices
pathsA1 = np.vstack((pathsS1[0], np.cumsum(pathsS1[1:], axis=0)/np.arange(1, 13)[:, np.newaxis])) # running averages
alive = np.full(npaths, True, dtype=bool)
payoffs = np.zeros(npaths, dtype=np.float)
for i in range(1, len(ts)):
    exerval = np.maximum(pathsA1[i]-K, 0)
    ind = ac_exer_or_cont(i, pathsA1[i], pathsS1[i]) & alive
    payoffs[ind] = exerval[ind]*np.exp(-r*ts[i])
    alive[ind] = False
V1_ = np.mean(payoffs)
V1_

5.3053756735085464

<b>Question 4(a) Upper Bound</b>

In [14]:
# nested simulation from time t_i when stock price is S
def ac_nested_mc(i, cumS, S, nsims=1000):
    alive = np.arange(nsims)                                   # keep track of indices of paths still alive
    payoff = np.zeros(nsims, dtype=np.float)
    pathsS = np.full(nsims, S, dtype=np.float)
    pathsA = np.full(nsims, cumS, dtype=np.float)
    for j in range(i+1, len(ts)):
        if len(alive) == 0:                                    # if exercised for all paths, stop
            break
        dt = ts[j]-ts[j-1]
        dW = np.random.randn(len(alive))*np.sqrt(dt)           # Brownian increment
        pathsS[alive] = pathsS[alive] * np.exp(-0.5*vol**2*dt+vol*dW)*np.exp((r-q)*dt)  # simulate S_{j+1} from S_j
        pathsA[alive] = (pathsA[alive]*(j-1) + pathsS[alive])/j    # running averages
        ind = ac_exer_or_cont(j, pathsA[alive], pathsS[alive])                    # identify the paths that need exercise
        payoff[alive[ind]] = np.maximum(pathsA[alive[ind]]-K, 0)*np.exp(-r*ts[j])      # update payoffs for exercised paths
        alive = alive[~ind]                                    # remove exercised paths 
    return np.mean(payoff)                                     # taking average of paths


# Upper bound
npaths = 1000                                                  # number of paths in the second independent run
nsims = 500                                                    # number of paths used in nested simulation
pathsS1 = blackscholes_mc(S0, vol, r, q, ts, npaths=npaths)                                  # stock prices
pathsA1 = np.vstack((pathsS1[0], np.cumsum(pathsS1[1:], axis=0)/np.arange(1, 13)[:, np.newaxis])) # running averages
pathsV1 = np.empty(pathsS1.shape, dtype=np.float)                 # discounted value process V_i (discounted to time zero)
pathsV1[0] = V1_                                                 # Initial value from lower bound simulation
pathsV1[-1] = np.maximum(pathsA1[-1]-K, 0)*np.exp(-r*ts[-1])      # values at final maturity
pathsEV1 = np.empty(paths.shape, dtype=np.float)                # Conditional expectation E[V_{i+1}|F_i], or continuation value
pathsEV1[0] = pathsV1[0]                                         # at time 0, option value = continuation value
for i in range(1, len(ts)-1):
    for j in range(npaths):
        S = pathsS1[i, j]                                        # stock price at time t_i and j-th path
        cumS = pathsA1[i, j]
        if ac_exer_or_cont(i, cumS, S):
            pathsV1[i, j] = np.maximum(cumS-K, 0)*np.exp(-r*ts[i]) # if exercised, V_i = exercise value
            pathsEV1[i, j] = ac_nested_mc(i, cumS, S, nsims=nsims)       # launch nested simulation to estimate E[V_{i+1}|F_i]
        else:
            pathsV1[i, j] = ac_nested_mc(i, cumS, S, nsims=nsims)        # if continue, use nested simulation to estimate V_i
            pathsEV1[i, j] = pathsV1[i, j]                       # E[V_{i+1}|F_i] = V_i
M1 = np.zeros_like(pathsS1)
M1[1:] = np.cumsum(pathsV1[1:]-pathsEV1[:-1], axis=0)             # martinglae increment V_{i+1}-E[V_{i+1}|F_i]
# taking average of max value of D_sF_s-M_s along each path
V1 = np.mean((np.maximum(pathsA1-K, 0)*np.exp(-r*ts[:, np.newaxis])-M1).max(axis=0))
V1

5.3954799105399953

<b>Question 4(b) Longstaff Schwartz Algorithm</b>

In [15]:
# Longstaff-Schwartz
S0, K, vol, r, q = 100, 100, 0.2, 0, 0
ts = np.linspace(0, 1, 13)
npaths = 100000
pathsS = blackscholes_mc(S0, vol, r, q, ts=ts, npaths=npaths)                                  # stock prices
pathsA = np.vstack((pathsS[0], np.cumsum(pathsS[1:], axis=0)/np.arange(1, 13)[:, np.newaxis])) # running averages
vol_ = 0.1
payoff = np.maximum(pathsA[-1]-K, 0)
betas_AC1 = np.full((len(ts), 6), np.nan, dtype=np.float)
for i in range(len(ts)-2, 0, -1):
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    payoff = payoff*discount
    A = np.vstack((np.ones_like(pathsS[i]), pathsA[i], pathsS[i], pathsA[i]**2, pathsS[i]**2, pathsA[i]*pathsS[i])).T
    betas_AC1[i] = np.linalg.lstsq(A, payoff)[0]           # regression to estimate continuation values
    contval=betas_AC1[i,0]+betas_AC1[i,1]*pathsA[i]+betas_AC1[i,2]*pathsS[i]+betas_AC1[i,3]*pathsA[i]**2+betas_AC1[i,4]*pathsS[i]**2+betas_AC1[i,5]*pathsA[i]*pathsS[i]
    exerval = np.maximum(pathsA[i]-K, 0)
    ind = exerval > contval                               # identify the paths where we should exercise
    payoff[ind] = exerval[ind]
betas_AC1

array([[             nan,              nan,              nan,
                     nan,              nan,              nan],
       [  1.57393991e+02,  -1.81856186e+00,  -1.81856186e+00,
          7.02371365e-03,   7.02371442e-03,   7.02371442e-03],
       [  1.55463990e+02,  -1.56719130e+00,  -2.02704446e+00,
          7.81742118e-03,   1.16402909e-02,   1.33147819e-03],
       [  1.51826069e+02,  -1.86829179e+00,  -1.64680926e+00,
          8.71899876e-03,   8.43211027e-03,   3.17477169e-03],
       [  1.49297677e+02,  -2.20631919e+00,  -1.25191408e+00,
          1.27157276e-02,   8.24986242e-03,  -9.85688499e-04],
       [  1.44909775e+02,  -2.40603086e+00,  -9.54317691e-01,
          1.40642527e-02,   6.72034669e-03,  -1.36944866e-03],
       [  1.39655386e+02,  -2.54479365e+00,  -7.01338411e-01,
          1.50973435e-02,   5.40351716e-03,  -1.72188481e-03],
       [  1.34290351e+02,  -2.64122851e+00,  -4.88378603e-01,
          1.62240409e-02,   4.65583521e-03,  -2.74673857e-03],


<b>Question 4(b) Lower Bound</b>

In [16]:
S0, K, vol, r, q, vol_ = 100, 100, 0.2, 0, 0, 0.1
ts = np.linspace(0, 1, 13)

# Optimal stopping strategy
def ac_exer_or_cont2(i,cumS, S):
    if i>=len(ts)-1:
        return cumS>K
    else:
        contval = betas_AC1[i,0]+betas_AC1[i,1]*cumS+betas_AC1[i,2]*S+betas_AC1[i,3]*cumS**2+betas_AC1[i,4]*S**2+betas_AC1[i,5]*cumS*S
        exerval = np.maximum(cumS-K, 0)
        return exerval>contval  # exercise only if exercise value is larger than continuation value

# Lower Bound
npaths = 1000000
pathsS2 = blackscholes_mc(S0, vol, r, q, ts, npaths=npaths)                                  # stock prices
pathsA2 = np.vstack((pathsS2[0], np.cumsum(pathsS2[1:], axis=0)/np.arange(1, 13)[:, np.newaxis])) # running averages
alive = np.full(npaths, True, dtype=bool)
payoffs = np.zeros(npaths, dtype=np.float)
for i in range(1, len(ts)):
    exerval = np.maximum(pathsA2[i]-K, 0)
    ind = ac_exer_or_cont2(i, pathsA2[i], pathsS2[i]) & alive
    payoffs[ind] = exerval[ind]*np.exp(-r*ts[i])
    alive[ind] = False
V2_ = np.mean(payoffs)
V2_

5.0865240507427547

<b>Question 4(b) Upper Bound</b>

In [17]:
# nested simulation from time t_i when stock price is S
def ac_nested_mc2(i, cumS, S, nsims=1000):
    alive = np.arange(nsims)                                   # keep track of indices of paths still alive
    payoff = np.zeros(nsims, dtype=np.float)
    pathsS = np.full(nsims, S, dtype=np.float)
    pathsA = np.full(nsims, cumS, dtype=np.float)
    for j in range(i+1, len(ts)):
        if len(alive) == 0:                                    # if exercised for all paths, stop
            break
        dt = ts[j]-ts[j-1]
        dW = np.random.randn(len(alive))*np.sqrt(dt)           # Brownian increment
        pathsS[alive] = pathsS[alive] * np.exp(-0.5*vol**2*dt+vol*dW)*np.exp((r-q)*dt)  # simulate S_{j+1} from S_j
        pathsA[alive] = (pathsA[alive]*(j-1) + pathsS[alive])/j    # running averages
        ind = ac_exer_or_cont2(j, pathsA[alive], pathsS[alive])                    # identify the paths that need exercise
        payoff[alive[ind]] = np.maximum(pathsA[alive[ind]]-K, 0)*np.exp(-r*ts[j])      # update payoffs for exercised paths
        alive = alive[~ind]                                    # remove exercised paths 
    return np.mean(payoff)                                     # taking average of paths


# Upper bound
npaths = 1000                                                  # number of paths in the second independent run
nsims = 500                                                    # number of paths used in nested simulation
pathsS2 = blackscholes_mc(S0, vol, r, q, ts, npaths=npaths)                                  # stock prices
pathsA2 = np.vstack((pathsS2[0], np.cumsum(pathsS2[1:], axis=0)/np.arange(1, 13)[:, np.newaxis])) # running averages
pathsV2 = np.empty(pathsS2.shape, dtype=np.float)                 # discounted value process V_i (discounted to time zero)
pathsV2[0] = V2_                                                 # Initial value from lower bound simulation
pathsV2[-1] = np.maximum(pathsA2[-1]-K, 0)*np.exp(-r*ts[-1])      # values at final maturity
pathsEV2 = np.empty(pathsS2.shape, dtype=np.float)                # Conditional expectation E[V_{i+1}|F_i], or continuation value
pathsEV2[0] = pathsV2[0]                                         # at time 0, option value = continuation value
for i in range(1, len(ts)-1):
    for j in range(npaths):
        S = pathsS2[i, j]                                        # stock price at time t_i and j-th path
        cumS = pathsA2[i, j]
        if ac_exer_or_cont2(i, cumS, S):
            pathsV2[i, j] = np.maximum(cumS-K, 0)*np.exp(-r*ts[i]) # if exercised, V_i = exercise value
            pathsEV2[i, j] = ac_nested_mc2(i, cumS, S, nsims=nsims)       # launch nested simulation to estimate E[V_{i+1}|F_i]
        else:
            pathsV2[i, j] = ac_nested_mc2(i, cumS, S, nsims=nsims)        # if continue, use nested simulation to estimate V_i
            pathsEV2[i, j] = pathsV2[i, j]                       # E[V_{i+1}|F_i] = V_i
M2 = np.zeros_like(pathsS2)
M2[1:] = np.cumsum(pathsV2[1:]-pathsEV2[:-1], axis=0)             # martinglae increment V_{i+1}-E[V_{i+1}|F_i]
# taking average of max value of D_sF_s-M_s along each path
V2 = np.mean((np.maximum(pathsA2-K, 0)*np.exp(-r*ts[:, np.newaxis])-M2).max(axis=0))
V2

5.4414743400560743

<b>Question 4(c) Duality Gaps</b>

In [18]:
dual_gap1 = {'Lower Bound': V1_ , 'Upper Bound': V1}  # Duality gap from (a)
dual_gap2 = {'Lower Bound': V2_ , 'Upper Bound': V2}  # Duality gap from (b)
print(dual_gap1)
print(dual_gap2)

{'Lower Bound': 5.3053756735085464, 'Upper Bound': 5.3954799105399953}
{'Lower Bound': 5.0865240507427547, 'Upper Bound': 5.4414743400560743}


<b>Question 4(c) Comment</b>
<ul> 
<li>The duality gap from (a) is more convergent than the duality gap from (b). Therefore, the basic functions in (a) do a better job than the basic functions in (b). Presumably the reason is that there exists overfitting in (b). 
</ul>
